<a href="https://colab.research.google.com/github/brendanpshea/programming_problem_solving/blob/main/Java_09_FilesExceptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Working with Villainous Data: Simple File Handling

At the *Bank of Evil*, nothing is more important than records. These are not just boring spreadsheets—they are the paper trail of world domination: deposits from stolen artwork, withdrawals for henchman payroll, and payments for custom shark tanks. If you can’t **store** this data somewhere permanent, you can’t run a proper evil empire.

### **Files and Why We Use Them**

A **file** is a place on your computer’s storage where data can live even after the program ends. You can think of it like the Bank of Evil’s vault: you can put things inside (writing) and take them out (reading).

Two common file formats for simple data are:

* **CSV (Comma-Separated Values)** – Each line represents one record, and values in that record are separated by commas.
* **JSON (JavaScript Object Notation)** – Data is stored in a way that looks like Java objects, but as plain text.

For now, we’ll start with **CSV** because it’s easier to work with as a beginner. A CSV file for villain bank transactions might look like this:

```
Villain,Amount
Gru,5000000
Scarlet Overkill,250000
Vector,1250000
```

### **Writing to a File in Java**

To put data into a file, Java has a class called **`FileWriter`**.

* A **class** is a blueprint for creating objects (you learned this in the OOP chapters).
* To use a class, we often need to create an **object** of that class.
* An object is created with the `new` keyword:

  ```java
  FileWriter writer = new FileWriter("transactions.csv");
  ```

  This line means: “Make a new FileWriter object, and connect it to a file called `transactions.csv`.”

The `FileWriter` class has a **method** called `write()` that adds text to the file:

```java
writer.write("Villain,Amount\n");
```

The `\n` means “start a new line” in the file.

When we’re done writing, we call the **`close()`** method to shut the connection:

```java
writer.close();
```

If we don’t close it, the data might not be saved properly.

One important note: **file writing can fail**. For example, you might not have permission to write to the folder. When that happens, Java will throw an **exception**—we’ll talk about those in detail later, but for now, we’ll use a small shortcut: adding

```java
throws IOException
```

to our method’s header tells Java, “I know this code might have a problem; if it does, pass it along to someone else to handle.”

Here’s a complete, short example of writing our villain transactions file:

In [2]:
%%writefile BankFileWrite.java
import java.io.FileWriter;
import java.io.IOException;

public class BankFileWrite {
    public static void main(String[] args) throws IOException {
        FileWriter writer = new FileWriter("transactions.csv");
        writer.write("Villain,Amount\n");
        writer.write("Gru,5000000\n");
        writer.write("Scarlet Overkill,250000\n");
        writer.write("Vector,1250000\n");
        writer.close();
    }
}

Writing BankFileWrite.java


In [3]:
!javac BankFileWrite.java
!java BankFileWrite

### **Reading a File in Java**

To read a file, we can use the **`Scanner`** class (you’ve used it before for keyboard input) with a **`File`** object.

1. A `File` object tells Java *where* the file is:

   ```java
   File file = new File("transactions.csv");
   ```
2. A `Scanner` can read the file line by line:

   ```java
   Scanner scanner = new Scanner(file);
   ```
3. We use `hasNextLine()` to check if there’s another line, and `nextLine()` to read it:

   ```java
   while (scanner.hasNextLine()) {
       String line = scanner.nextLine();
       System.out.println(line);
   }
   ```

Like with writing, reading a file can fail if the file isn’t there—so here we say:

```java
throws FileNotFoundException
```

to let Java know we’re aware.

Here’s the full example:


In [1]:
%%writefile BankFileRead.java
import java.io.File;
import java.io.FileNotFoundException;
import java.util.Scanner;

public class BankFileRead {
    public static void main(String[] args) throws FileNotFoundException {
        File file = new File("transactions.csv");
        Scanner scanner = new Scanner(file);

        while (scanner.hasNextLine()) {
            String line = scanner.nextLine();
            System.out.println(line);
        }
        scanner.close();
    }
}


Writing BankFileRead.java


In [4]:
!javac BankFileRead.java
!java BankFileRead

Villain,Amount
Gru,5000000
Scarlet Overkill,250000
Vector,1250000



### What We've Learned
In this section you’ve learned:

* What files are and why we use them.
* How to write text to a CSV file with `FileWriter`.
* How to read text from a CSV file with `Scanner` and `File`.
* That file operations can fail, leading to **exceptions**, which we’ll dig into next.



## Expecting the Unexpected: What Are Exceptions?

At the Bank of Evil, the unexpected is part of daily life. Henchmen may forget to lock the vault. Sharks in the lair’s moat may refuse to eat the intruder. And sometimes, the “Deposit All Loot” button mysteriously wires funds to the *Bank of Goodness* instead. In programming, unexpected things happen too, and when they do, Java uses something called an **exception** to let us know.

### What an exception is

An **exception** is a special object that represents a problem in your program. When something goes wrong—like trying to open a file that doesn’t exist—Java *throws* an exception. Throwing an exception means the program stops its normal flow and looks for someone who can handle the problem.

If no one handles the exception, the program will crash. In the real world, that’s like a bank teller running away screaming instead of helping you fix a transaction.

### Common examples

Here are a few things that can cause exceptions:

* Trying to read a file that isn’t there.
* Dividing by zero.
* Asking a program to convert `"banana"` into a number.
* Accessing an array at an index that doesn’t exist.

### Checked vs. unchecked exceptions

In Java, there are two main kinds:

* **Checked exceptions** – Problems Java makes you deal with at compile time. For example, reading a file can throw a `FileNotFoundException`. You either have to handle it with a `try…catch` block or declare it with `throws`.
* **Unchecked exceptions** – Problems that can happen while running your program (runtime errors) and that Java doesn’t force you to handle, like dividing by zero.

For beginners, the main thing to remember is: **checked exceptions require handling**.

### Throwing and catching exceptions

If something goes wrong, Java will “throw” an exception. If your code knows how to deal with it, you can “catch” it and respond, so the whole program doesn’t crash.

We’ll see how to do that with `try…catch` in the next section. For now, here’s a very small example that will deliberately cause a `FileNotFoundException`:

```java
import java.io.File;
import java.io.FileNotFoundException;
import java.util.Scanner;

public class MissingFileExample {
    public static void main(String[] args) throws FileNotFoundException {
        File file = new File("nonexistent.csv");
        Scanner scanner = new Scanner(file); // This will throw an exception
    }
}
```

In this code:

1. We create a `File` object pointing to a file that doesn’t exist.
2. When `new Scanner(file)` tries to read it, Java throws a `FileNotFoundException`.
3. Since we wrote `throws FileNotFoundException` in `main`, we told Java to pass the problem upward (to the program’s top level), which will crash and print an error message.

That error message might look something like this:

```
Exception in thread "main" java.io.FileNotFoundException: nonexistent.csv (No such file or directory)
    at java.base/java.util.Scanner.<init>(Scanner.java:639)
    ...
```

The message includes:

* The type of exception (`FileNotFoundException`).
* A description of what went wrong.
* A “stack trace” showing where in the code it happened.

### Why exceptions matter

Ignoring exceptions is like ignoring alarms in the bank vault. You might get away with it once, but sooner or later, disaster strikes. Exceptions let your program notice problems and recover from them, or at least fail in a controlled way.

In the next section, we’ll see how to actually **catch** exceptions with `try…catch` so that the Bank of Evil’s systems can keep running even when something goes wrong.
